## IMPORT REQUIRED LIBRARIES

In [69]:

import pandas as pd
from datetime import datetime
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
import plotly.express as px
import warnings
warnings.filterwarnings('ignore')

## LOAD THE DATA

In [70]:
url = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol=RELIANCE.BSE&outputsize=full&apikey=AUPPYZ3PTCAM6OO3&datatype=csv'
data = pd.read_csv(url)

#### We used the API provided by www.alphavantage.co and imported it using pandas

## DATA DESCRIPTION

In [71]:
data.head()

,timestamp,open,high,low,close,adjusted_close,volume,dividend_amount,split_coefficient
0,2023-03-27,2215.5000,2254.0000,2200.50,2237.5000,2237.5000,315446,0.0,1.0
1,2023-03-24,2247.6001,2250.0000,2198.00,2203.5000,2203.5000,178216,0.0,1.0
2,2023-03-23,2270.0000,2270.0000,2243.75,2247.5500,2247.5500,208447,0.0,1.0
3,2023-03-22,2280.1001,2291.3000,2270.00,2276.6499,2276.6499,230717,0.0,1.0
4,2023-03-21,2222.0000,2273.8501,2218.00,2270.0500,2270.0500,441746,0.0,1.0


In [72]:
data.describe()

,open,high,low,close,adjusted_close,volume,dividend_amount,split_coefficient
count,4494.000000,4494.000000,4494.000000,4494.000000,4494.000000,4.494000e+03,4494.000000,4494.000000
mean,1332.183349,1349.989718,1313.831924,1331.327741,804.289234,3.104615e+06,0.018539,1.000447
std,619.586969,629.267389,609.890287,619.519153,709.727968,2.117898e+07,0.311301,0.021094
min,377.517600,381.517100,372.434300,374.826500,80.531600,4.245200e+04,0.000000,1.000000
25%,893.774700,903.637200,883.049700,892.774700,383.284875,4.540135e+05,0.000000,1.000000
50%,1064.999700,1078.749650,1051.174700,1064.549650,471.806300,9.345640e+05,0.000000,1.000000
75%,1769.999400,1802.824900,1745.249425,1775.061975,1068.411150,2.692082e+06,0.000000,1.000000
max,3215.999000,3252.099100,3135.249100,3216.299000,2810.735400,8.021841e+08,8.000000,2.000000


## PREPARING THE DATA TO FIT INTO THE MODEL

In [73]:
data = data.drop(['adjusted_close'], axis=1)
data['timestamp'] = data['timestamp'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d').toordinal())

## SPLIT INTO INPUT AND TARGET

In [74]:
X = data.drop(['close'], axis=1)
y = data['close']

## USING GRIDSEARCHCV TO FIND THE BEST PARAMETERS

In [75]:
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [5, 10, 15],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

In [ ]:
rf = RandomForestRegressor()

In [76]:
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, n_jobs=-1)
grid_search.fit(X, y)

GridSearchCV(cv=5, estimator=RandomForestRegressor(max_depth=15), n_jobs=-1,
             param_grid={'max_depth': [5, 10, 15],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [100, 200, 300]})

## FITTING THE MODEL

In [77]:
rf = RandomForestRegressor(n_estimators=grid_search.best_params_['n_estimators'],
                            max_depth=grid_search.best_params_['max_depth'],
                            min_samples_split=grid_search.best_params_['min_samples_split'],
                            min_samples_leaf=grid_search.best_params_['min_samples_leaf'])

rf.fit(X, y)

RandomForestRegressor(max_depth=15, n_estimators=300)

## PLOTTING PREDICTED AGAINST THE ACTUAL DATA

In [78]:
y_pred = rf.predict(X)
data['timestamp'] = data['timestamp'].apply(lambda x: datetime.fromordinal(x))
df = pd.DataFrame({'timestamp': data['timestamp'], 'actual': y, 'predicted': y_pred})
fig = px.line(df, x='timestamp', y=['actual', 'predicted'], title='Actual vs Predicted Close')
fig.show()

## PREDICTING FOR A RANDOM VALUE

In [79]:
ordinal = datetime.strptime('2023-03-27', '%Y-%m-%d').toordinal()
predicted_close = rf.predict([[ordinal, data['open'][0], data['high'][0], data['low'][0], data['volume'][0], data['dividend_amount'][0], data['split_coefficient'][0]]])[0]
print('Predicted Close:', predicted_close,ordinal)

actual_close = data.iloc[0]['close']
print('Actual Close:', actual_close)

Predicted Close: 2236.1539665357136 738606
Actual Close: 2237.5
